In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

import tensorflow as tf
import kerastuner as kt
from keras_tuner import BayesianOptimization
from tensorflow.keras.callbacks import  EarlyStopping

from sae_dnn_model import sae_model, dnn_model

2022-09-14 10:44:58.007501: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-14 10:44:58.181630: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-09-14 10:44:58.253316: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-14 10:44:58.849297: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [3]:
X = pd.read_csv('dataset/df_pubchem_rapi.csv')
X

,0,1,2,3,4,5,6,7,8,9,...,871,872,873,874,875,876,877,878,879,880
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5645,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5646,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5647,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5648,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
Y = pd.read_csv('dataset/kelas_data.csv')
Y = np.array(Y)

In [5]:
#Fungsi model untuk tuning
def build_model(hp):
    #Isi parameter yang akan dituning
    params = {
              'hl_node' : hp.Choice('units',values= para_hl_node),
              'af' : hp.Choice('activation',values= para_af),
              'lr' : hp.Choice('learning_rate',values= para_lr),
              'opt' : hp.Choice('optimizer',values= para_opt),
              'num_layers' : hp.Choice('num_layers',values= para_num_layers),
              'do' : hp.Choice('dropout_rate',values= para_do),
              'fr_node' : hp.Choice('fraction_node',values= para_fr_node)
              }
    #Latih model SAE
    sae_weights = sae_model(xt = X, xv = X_train, EPOCHS= 100,**params)
    #Latih model DNN dengan bobot SAE
    sae_dnn = dnn_model(X_train, sae_weights=sae_weights, EPOCHS= 100,**params)
    return sae_dnn

In [6]:
para_hl_node = [320, 640, 1280, 1500, 1600, 1700, 1800]
para_af = ["relu"]
para_lr = [x for x in np.linspace(0.01,0.1)]
para_opt = ["adam", "adagrad"]
para_num_layers = [2,3,4,5]
para_do = [0.5,0.6,0.7,0.8]
para_fr_node = [0.5,0.66,0.75]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2)
#hypterparameter tuning
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_recall',patience = 25)

# Fungsi Bayesian Optimization di Keras Tuner.
tuner = BayesianOptimization(build_model,
    # Metrik yang dicari optimalnya
    objective= kt.Objective("val_recall", direction="max"), 
    # Jumlah percobaan
    max_trials=2,
    executions_per_trial=2,
    # Folder simpan hasil tuning
    directory='tuning-model',
    project_name='sae_dnn_tuning_bayesFiltered40k', overwrite = True)

#Jalankan Keras Tuner u
tuner.search(X_train, y_train, epochs=100, validation_data=(X_test, y_test),callbacks=[stop_early])
#Tampilkan hasil terbaik
tuner.results_summary()

Trial 2 Complete [00h 02m 10s]
val_recall: 0.8342975080013275

Best val_recall So Far: 0.839669406414032
Total elapsed time: 00h 06m 59s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in tuning-model/sae_dnn_tuning_bayesFiltered40k
Showing 10 best trials
Trial summary
Hyperparameters:
units: 1800
activation: relu
learning_rate: 0.030204081632653063
optimizer: adagrad
num_layers: 4
dropout_rate: 0.5
fraction_node: 0.5
Score: 0.839669406414032
Trial summary
Hyperparameters:
units: 1800
activation: relu
learning_rate: 0.015510204081632653
optimizer: adam
num_layers: 4
dropout_rate: 0.8
fraction_node: 0.75
Score: 0.8342975080013275
